In [1]:
import sys
sys.executable

'/usr/local/opt/python/bin/python3.7'

In [2]:
import numpy as np
import pandas as pd
from scipy.spatial import KDTree
from scipy.stats import uniform, expon, poisson, describe
import math

import matplotlib.pyplot as plt

from copy import deepcopy
from collections import defaultdict

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import pickle

In [3]:
def run_simulation(seed, process=1):
    np.random.seed(seed) 
    
    results = defaultdict(lambda: float('inf'))
    results['seed'] = seed
    
    X = [x_init[0]]
    Y = [x_init[1]]
    
    lmbda = (n * (r(n, D) ** 2) * math.pi)

    R = [None]
    Theta = [None]
    S = [None]
    Gamma = [0]

    t = 0
    while np.linalg.norm(np.array([X[-1], Y[-1]]) - x_goal) > r(n, D) and t < 2000:
        t += 1 # timestep index

        # three samples for each time step
        R.append(r(n, D) * (np.random.uniform() ** 0.5))
        S.append(np.random.choice(a=[-1, 1]))
        if process == 1:
            Theta.append((math.pi / lmbda) * np.random.exponential())   # uses asymptotics on the # of points in a ball explicitly
        elif process == 2:
            n_B = np.random.poisson((r(n, D) ** 2) * math.pi * n)       # simulates the number of points in a ball, leading to a mixture of exponentials
            Theta.append((math.pi / n_B) * np.random.exponential())
                                   
        # now we can determine the rest
        X.append(X[t-1] + R[t] * np.cos(Gamma[t-1] - Theta[t] * S[t]))
        Y.append(Y[t-1] + R[t] * np.sin(Gamma[t-1] - Theta[t] * S[t]))

        g = np.arcsin((R[t] / np.linalg.norm(np.array([X[t], Y[t]]) - x_goal)) * np.sin(Theta[t] * S[t]))
        Gamma.append(Gamma[t-1] + g)
        
    if t < 1000:
        results['T'] = t
        results['length'] = sum(R[1:])
        results['last_point'] = (X[-1], Y[-1])
        results['distance_to_goal'] = np.linalg.norm(np.array([X[-1], Y[-1]]) - x_goal) 
        results['R'] = R
        results['Theta'] = Theta
        results['S'] = S
        results['X'] = X
        results['Y'] = Y
        
    return results

def run_simulation_rej(seed, process=1):
    np.random.seed(seed) 
    
    results = defaultdict(lambda: float('inf'))
    results['seed'] = seed
    
    X = [x_init[0]]
    Y = [x_init[1]]
    
    lmbda = (n * (r(n, D) ** 2) * math.pi)
    
    # WARNING: note the 0-index entries! these are not actual data and will mess with the statistics
    R = [r(n, D)]
    Theta = [0]
    S = [0]
    Gamma = [0]

    t = 0
    while np.linalg.norm(np.array([X[-1], Y[-1]]) - x_goal) > r(n, D) and t < 2000:
        t += 1 # timestep index

        # three samples for each time step
        # rejection sampling (to account for dependence between balls of adjacent timesteps)
        reject = True
        while reject:
            r_sample = r(n, D) * (np.random.uniform() ** 0.5)
            s_sample = np.random.choice(a=[-1, 1])
            if process == 1:
                th_sample = (math.pi / lmbda) * np.random.exponential()       # uses asymptotics on the # of points in a ball explicitly
            elif process == 2:
                n_B = np.random.poisson((r(n, D) ** 2) * math.pi * n)         # simulates the number of points in a ball, leading to a mixture of exponentials
                th_sample = (math.pi / n_B) * np.random.exponential()
            
            # reject if (r_next < r(n)-r_now AND theta_next < theta_now)
            if not (r_sample < r(n, D) - R[-1] and th_sample < Theta[-1]):
                reject = False
            
        R.append(r_sample)
        S.append(s_sample)
        if process == 1:
            Theta.append(th_sample)   
        elif process == 2:   
            Theta.append(th_sample)
                                   
        # now we can determine the rest
        X.append(X[t-1] + R[t] * np.cos(Gamma[t-1] - Theta[t] * S[t]))
        Y.append(Y[t-1] + R[t] * np.sin(Gamma[t-1] - Theta[t] * S[t]))

        g = np.arcsin((R[t] / np.linalg.norm(np.array([X[t], Y[t]]) - x_goal)) * np.sin(Theta[t] * S[t]))
        Gamma.append(Gamma[t-1] + g)
        
    if t < 1000:
        results['T'] = t
        results['length'] = sum(R[1:])
        results['last_point'] = (X[-1], Y[-1])
        results['distance_to_goal'] = np.linalg.norm(np.array([X[-1], Y[-1]]) - x_goal) 
        results['R'] = R
        results['Theta'] = Theta
        results['S'] = S
        results['X'] = X
        results['Y'] = Y
        
    return results

In [10]:
n = 5000000
D = 2

ell = 0.8 * np.sqrt(2)

x_init = np.array([0, 0])
x_goal = np.array([ell, 0])

# r_n function
def r(n, D):
    return (n ** (-1/(2*D))) / 5
    # designed for n = 5 * 10^6

#############

simulation_outputs = dict()     # key, value = process, dict

for i in range(1, 3):
    simulation_outputs[str(i)] = defaultdict(list)
    simulation_outputs[str(i) + '-rej'] = defaultdict(list)
    for j in range(1, 1001):
        simulation_outputs[str(i)][j] = run_simulation(j * 10, process=i)
        simulation_outputs[str(i) + '-rej'][j] = run_simulation_rej(j * 10, process=i)

#############        

lengths_free = defaultdict(list)
counts_free = defaultdict(list)
asymp_free = defaultdict(list)

for k1 in simulation_outputs.keys():
    for k2 in simulation_outputs[k1].keys():
        lengths_free[k1].append(simulation_outputs[k1][k2]['length'])
        counts_free[k1].append(simulation_outputs[k1][k2]['T'])
        asymp_free[k1].append(simulation_outputs[k1][k2]['length'] * (simulation_outputs[k1][k2]['T'] ** 0.5))
        
const = ell - r(n, D)

# statistics on length

#print(describe(lengths_based - const))
for k1 in simulation_outputs.keys():
    print(str(k1))    
    print(describe(lengths_free[k1] - const))
    print(' ')
    
# statistics on T

#print(describe(counts_based))
for k1 in simulation_outputs.keys():
    print(str(k1))    
    print(describe(counts_free[k1]))
    print(' ')
    
# statistics on asymp

#print(describe(asymp_based))
for k1 in simulation_outputs.keys():
    print(str(k1))    
    print(describe(asymp_free[k1]))
    print(' ')

1
DescribeResult(nobs=1000, minmax=(0.0001345144988875724, 0.00428463147208058), mean=0.00173465986636877, variance=1.038372196513653e-06, skewness=0.36409614780069244, kurtosis=-0.8271217192521689)
 
1-rej
DescribeResult(nobs=1000, minmax=(0.00013020313858347343, 0.004282828158892471), mean=0.0017762423158081487, variance=9.99660767284966e-07, skewness=0.28922721884661234, kurtosis=-0.840580057086715)
 
2
DescribeResult(nobs=1000, minmax=(0.00012302208773662393, 0.004234521236498923), mean=0.0017721768229152807, variance=1.0962342577870154e-06, skewness=0.3305586120207213, kurtosis=-0.9050204382539513)
 
2-rej
DescribeResult(nobs=1000, minmax=(0.00014756185048314663, 0.004339574776342392), mean=0.0017604000465535525, variance=1.0518520497141016e-06, skewness=0.39819881251501155, kurtosis=-0.7802438590245697)
 
1
DescribeResult(nobs=1000, minmax=(373, 423), mean=400.207, variance=52.668819819819824, skewness=-0.09933534510938041, kurtosis=-0.04723805011446913)
 
1-rej
DescribeResult(no

In [11]:
n = 1000000
D = 2

ell = 0.8 * np.sqrt(2)

x_init = np.array([0, 0])
x_goal = np.array([ell, 0])

# r_n function
def r(n, D):
    return (n ** (-1/(2*D))) / 10
    # designed for n = 10^6

#############

simulation_outputs = dict()     # key, value = process, dict

for i in range(1, 3):
    simulation_outputs[str(i)] = defaultdict(list)
    simulation_outputs[str(i) + '-rej'] = defaultdict(list)
    for j in range(1, 1001):
        simulation_outputs[str(i)][j] = run_simulation(j * 10, process=i)
        simulation_outputs[str(i) + '-rej'][j] = run_simulation_rej(j * 10, process=i)

#############        

lengths_free = defaultdict(list)
counts_free = defaultdict(list)
asymp_free = defaultdict(list)

for k1 in simulation_outputs.keys():
    for k2 in simulation_outputs[k1].keys():
        lengths_free[k1].append(simulation_outputs[k1][k2]['length'])
        counts_free[k1].append(simulation_outputs[k1][k2]['T'])
        asymp_free[k1].append(simulation_outputs[k1][k2]['length'] * (simulation_outputs[k1][k2]['T'] ** 0.5))
        
const = ell - r(n, D)

# statistics on length

#print(describe(lengths_based - const))
for k1 in simulation_outputs.keys():
    print(str(k1))    
    print(describe(lengths_free[k1] - const))
    print(' ')
    
# statistics on T

#print(describe(counts_based))
for k1 in simulation_outputs.keys():
    print(str(k1))    
    print(describe(counts_free[k1]))
    print(' ')
    
# statistics on asymp

#print(describe(asymp_based))
for k1 in simulation_outputs.keys():
    print(str(k1))    
    print(describe(asymp_free[k1]))
    print(' ')

1
DescribeResult(nobs=1000, minmax=(0.008687432973464304, 0.01810107889298429), mean=0.01262592877630674, variance=2.0831954417861454e-06, skewness=0.3089137593170177, kurtosis=0.20009361829357264)
 
1-rej
DescribeResult(nobs=1000, minmax=(0.009240954024080539, 0.01841057706423599), mean=0.01323452164057285, variance=2.2726836696233563e-06, skewness=0.38844391582547955, kurtosis=0.14582178872297513)
 
2
DescribeResult(nobs=1000, minmax=(0.008641595921087708, 0.019699984973848172), mean=0.013887866134450961, variance=2.7052413477219267e-06, skewness=0.22370428171610285, kurtosis=0.16809009279741716)
 
2-rej
DescribeResult(nobs=1000, minmax=(0.00965044002982518, 0.021299277956816853), mean=0.014530889908432447, variance=2.7904865132953882e-06, skewness=0.26042871423100444, kurtosis=0.18732464299720997)
 
1
DescribeResult(nobs=1000, minmax=(509, 566), mean=540.94, variance=68.70110110110113, skewness=-0.020893967121893637, kurtosis=0.10840834392858278)
 
1-rej
DescribeResult(nobs=1000, mi

In [12]:
n = 500000
D = 2

ell = 0.8 * np.sqrt(2)

x_init = np.array([0, 0])
x_goal = np.array([ell, 0])

# r_n function
def r(n, D):
    return (n ** (-1/(2*D))) /2
    # designed for n = 500000

#############

simulation_outputs = dict()     # key, value = process, dict

for i in range(1, 3):
    simulation_outputs[str(i)] = defaultdict(list)
    simulation_outputs[str(i) + '-rej'] = defaultdict(list)
    for j in range(1, 1001):
        simulation_outputs[str(i)][j] = run_simulation(j * 10, process=i)
        simulation_outputs[str(i) + '-rej'][j] = run_simulation_rej(j * 10, process=i)

#############        

lengths_free = defaultdict(list)
counts_free = defaultdict(list)
asymp_free = defaultdict(list)

for k1 in simulation_outputs.keys():
    for k2 in simulation_outputs[k1].keys():
        lengths_free[k1].append(simulation_outputs[k1][k2]['length'])
        counts_free[k1].append(simulation_outputs[k1][k2]['T'])
        asymp_free[k1].append(simulation_outputs[k1][k2]['length'] * (simulation_outputs[k1][k2]['T'] ** 0.5))
        
const = ell - r(n, D)

# statistics on length

#print(describe(lengths_based - const))
for k1 in simulation_outputs.keys():
    print(str(k1))    
    print(describe(lengths_free[k1] - const))
    print(' ')
    
# statistics on T

#print(describe(counts_based))
for k1 in simulation_outputs.keys():
    print(str(k1))    
    print(describe(counts_free[k1]))
    print(' ')
    
# statistics on asymp

#print(describe(asymp_based))
for k1 in simulation_outputs.keys():
    print(str(k1))    
    print(describe(asymp_free[k1]))
    print(' ')

1
DescribeResult(nobs=1000, minmax=(5.280583179545495e-05, 0.01850947592742247), mean=0.006929341065037597, variance=2.0842106867664747e-05, skewness=0.4563956724487127, kurtosis=-0.7644932801474971)
 
1-rej
DescribeResult(nobs=1000, minmax=(4.79547687679549e-05, 0.01851137659059021), mean=0.007353426248397164, variance=2.1518632984879874e-05, skewness=0.33035121951098406, kurtosis=-0.81629061361491)
 
2
DescribeResult(nobs=1000, minmax=(3.7000341433834905e-05, 0.017976914993816262), mean=0.0069255549128103425, variance=2.037552335990687e-05, skewness=0.4500273480622023, kurtosis=-0.7151250508651565)
 
2-rej
DescribeResult(nobs=1000, minmax=(2.769114088763125e-05, 0.018313903539687093), mean=0.007382073074168936, variance=2.1349242597898637e-05, skewness=0.3409012024368316, kurtosis=-0.8047160509341715)
 
1
DescribeResult(nobs=1000, minmax=(79, 100), mean=89.309, variance=11.208727727727728, skewness=0.1273927686379826, kurtosis=-0.16853951671924694)
 
1-rej
DescribeResult(nobs=1000, m